In [1]:
import json
from data.preprocess import DataProcessor
import os
from data.custom_dataset import CustomDataset
from data.utils import get_dataset, get_tokenizer, get_dataloader
from model.model import Model
import torch
import pandas as pd
import numpy as np
import torch.optim as optim
import ast
import os
import itertools
import torch.utils.checkpoint
from torch.cuda.amp import GradScaler, autocast
from training.trainer import Trainer
import argparse

/vol/bitbucket/mh1022/dl_cw_pyenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
# parser = argparse.ArgumentParser(description="Train model",
#                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
# parser.add_argument("-n", "--num_chunks", type=int, help="number of chunks")
# parser.add_argument("-r", "--run_name", type=str, help="run name")
# parser.add_argument("-m", "--max_epochs", type=int, help="number of max epochs")
# parser.add_argument("-l", "--num_heads_labattn", type=int, help="number of heads for lab attention")
# parser.add_argument("-p", "--patience_threshold", type=int, help="patience threshold")

# args = parser.parse_args()
# args_config = vars(args)

# # device
USE_GPU = True
dtype = torch.float32
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
cpu = torch.device('cpu')
print(device)

### for debugging use cpu
#device = cpu

config = {
#    "run_name": "Run_test_TLWAN"
    "run_name": "test"
    ,"project_path": '/vol/bitbucket/mh1022/temporal-modelling-icd'
    ,"base_checkpoint": os.path.join("", "RoBERTa-base-PM-M3-Voc-hf")
    ,"num_attention_heads": 1
    ,"num_layers": 1
    ,"lr": 5e-5
    ,"max_chunks": 16
    ,"grad_accumulation": 16
    ,"use_positional_embeddings": True
    ,"use_reverse_positional_embeddings": True
    ,"priority_mode": "None"
    ,"priority_idxs": [1]
    ,"use_document_embeddings": True
    ,"use_reverse_document_embeddings": True
    ,"use_category_embeddings": True
    ,"num_labels": 50
    ,"use_all_tokens": True
    ,"num_heads_labattn": 1
    ,"final_aggregation": "cls"
    ,"only_discharge_summary": False
    ,"patience_threshold": 3
    ,"max_epochs": 20
    ,"save_model": False
    ,"load_from_checkpoint": False
    ,"checkpoint_name": "Run_all_notes_last_second_transf"
    ,"evaluate_temporal": False
    ,"use_multihead_attention": False
    ,"debug": False
}
with open(os.path.join("", f"results/config_{config['run_name']}.json"), "w") as f:
    json.dump(config, f)

# process and aggregate raw data
dp = DataProcessor(dataset_path="dataset", config=config)
notes_agg_df = dp.aggregate_data()

# get tokenizer
tokenizer = get_tokenizer(config["base_checkpoint"])

# Get training / validation / test
dataset_config = {
    "max_chunks" : config["max_chunks"],
    "priority_mode" : config["priority_mode"],
    "priority_idxs" : config["priority_idxs"]
}
training_set = get_dataset(notes_agg_df, "TRAIN", tokenizer = tokenizer, **dataset_config)
training_generator = get_dataloader(training_set)

validation_set =  get_dataset(notes_agg_df, "VALIDATION", tokenizer = tokenizer, **dataset_config)
validation_generator = get_dataloader(validation_set)

test_set = get_dataset(notes_agg_df, "TEST", tokenizer = tokenizer, **dataset_config)
test_generator = get_dataloader(test_set)


cuda:0


/vol/bitbucket/mh1022/temporal-modelling-icd/data/preprocess.py:16: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  self.notes_df = pd.read_csv(os.path.join(dataset_path, "NOTEEVENTS.csv"))


{'Nursing/other': 0, 'Radiology': 1, 'Nursing': 2, 'Physician ': 3, 'ECG': 4, 'Discharge summary': 5, 'Respiratory ': 6, 'Echo': 7, 'Nutrition': 8, 'General': 9, 'Rehab Services': 10, 'Social Work': 11, 'Case Management ': 12, 'Consult': 13, 'Pharmacy': 14}


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.metrics
import torch
import torch.nn as nn
import transformers
import torch.nn.functional as F
import tqdm as tqdm
import torch.optim as optim
import ast
import os
import itertools
from torch.utils.data import Dataset
import torch.utils.checkpoint
import random
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from evaluation.metrics import MyMetrics
from evaluation.evaluate import evaluate
from data.custom_dataset import OneSampleDataset
import ipdb
from torch.utils.data import DataLoader

from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoModel,
)


In [ ]:
for t, sample in enumerate(tqdm(training_generator)):
    # First pass through the model: obtain largest gradients
    one_sample_dataset = OneSampleDataset(sample)
    one_sample_dataloader = DataLoader(one_sample_dataset, batch_size=config["max_chunks"], shuffle=False)
    for j, data in enumerate(tqdm(one_sample_dataloader)):
        labels = data["label"][0][: 50]
        input_ids = data["input_ids"][0]
        attention_mask = data["attention_mask"][0]
        seq_ids = data["seq_ids"][0]
        category_ids = data["category_ids"][0]
        # note_end_chunk_ids = data["note_end_chunk_ids"]
        cutoffs = data["cutoffs"]
        print(input_ids.shape)


In [10]:
data["input_ids"].shape

torch.Size([1, 512])

In [11]:
sample["input_ids"].shape

torch.Size([1, 38, 512])

In [ ]:
for t, sample in enumerate(tqdm(training_generator)):
    # First pass through the model: obtain largest gradients
    one_sample_dataset = OneSampleDataset(sample)
    one_sample_dataloader = DataLoader(one_sample_dataset, batch_size=self.config["max_chunks"], shuffle=False)
    for j, data in enumerate(tqdm(one_sample_dataloader)):
        labels = data["label"][0][: self.model.num_labels]
        input_ids = data["input_ids"][0]
        attention_mask = data["attention_mask"][0]
        seq_ids = data["seq_ids"][0]
        category_ids = data["category_ids"][0]
        # note_end_chunk_ids = data["note_end_chunk_ids"]
        cutoffs = data["cutoffs"]

In [69]:
input_ids

tensor([[    0,    54,  5478,  ...,     1,     1,     1],
        [    0,    68, 47179,  ...,     1,     1,     1],
        [    0,  9807,    56,  ...,     1,     1,     1],
        [    0,  3009,  2137,  ...,    17, 16122,     2],
        [    0,  7714, 22631,  ...,     1,     1,     1]])

In [56]:
sample = training_set[0]
one_sample_dataset = OneSampleDataset(sample)
one_sample_data_loader =DataLoader(one_sample_dataset, batch_size=4)

In [52]:
sample["input_ids"].shape

torch.Size([15, 512])

In [60]:
one_sample_dataset[0]["input_ids"].shape

torch.Size([512])

In [59]:
next(iter(one_sample_data_loader))["input_ids"].shape

torch.Size([4, 512])

In [45]:
from tqdm import tqdm

In [57]:
for j, data in enumerate(tqdm(one_sample_data_loader)):
    labels = data["label"][0][: 50]
    input_ids = data["input_ids"][0]
    attention_mask = data["attention_mask"][0]
    seq_ids = data["seq_ids"][0]
    category_ids = data["category_ids"][0]
    # note_end_chunk_ids = data["note_end_chunk_ids"]
    cutoffs = data["cutoffs"]
    print(input_ids.shape)


100%|██████████| 4/4 [00:00<00:00, 1351.58it/s]

torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])


In [37]:
token_encodings = torch.rand((4, 512, 768))


In [38]:
token_encodings.requires_grad = True

In [40]:
loss = token_encodings.sum()

In [41]:
loss.backward()

In [42]:
token_encodings.grad

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 

In [34]:
first_batch["input_ids"].shape

torch.Size([4, 512])

In [21]:
sample["seq_ids"][0]

tensor(0)

In [23]:
sample["input_ids"][0,:].shape

torch.Size([512])